In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [10]:
data = pd.read_excel('brazilian credit card.xlsx')
selected_vars =['cep_cat numeric',
 'score_cat numeric',
 'velocidade_cat numeric',
 'bandeira_cat numeric',
 'qtde_parc_cat numeric',
 'valor_trans_cat numeric',
 'trans_nacional_cat numeric',
 'mcc_cat numeric',
 'valor_trans_ant_cat numeric',
 'mcc_ant_cat numeric',
 'dif_score_cat numeric',
 'cep_ant_cat numeric',
 'lim_cred_cat numeric',
 'Class']

In [11]:
tr_data = data[selected_vars]
tr_data.shape

(374823, 14)

In [12]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [13]:
y=tr_data['Class']

In [14]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [15]:
X_train0,X_tset,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.5, shuffle=False,random_state=0)

from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.013,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)


df1_normal=X_train1[y_train1==0]
clf = SGDOneClassSVM(nu=0.013,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

In [16]:
X_train0.shape,X_train1.shape

((116325, 13), (126053, 13))

In [17]:
y_train0.shape,y_train1.shape

((116325,), (126053,))

In [18]:
X_tset.shape

(112447, 13)

# N=1

In [19]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_tset.to_numpy()

## Reshape input to be 3D [samples, timesteps, features] (format requis par LSTM)
train_LSTM_X0 = train_LSTM_X0.reshape((train_LSTM_X0.shape[0], 1, train_LSTM_X0.shape[1]))
test_LSTM_X = test_LSTM_X.reshape((test_LSTM_X.shape[0], 1, test_LSTM_X.shape[1]))

## Recuperation des labels
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [20]:
inputs=Input((1,13))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [21]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [22]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=500,batch_size=200)

Epoch 1/500
582/582 [==============================] - 5s 3ms/step - loss: 0.2164 - precision: 0.1627 - recall: 0.0805
Epoch 2/500
582/582 [==============================] - 2s 4ms/step - loss: 0.1158 - precision: 0.5626 - recall: 0.0951
Epoch 3/500
582/582 [==============================] - 2s 3ms/step - loss: 0.1124 - precision: 0.5815 - recall: 0.0958
Epoch 4/500
582/582 [==============================] - 2s 4ms/step - loss: 0.1101 - precision: 0.5980 - recall: 0.1096
Epoch 5/500
582/582 [==============================] - 2s 3ms/step - loss: 0.1081 - precision: 0.6084 - recall: 0.1219
Epoch 6/500
582/582 [==============================] - 2s 4ms/step - loss: 0.1064 - precision: 0.6025 - recall: 0.1303
Epoch 7/500
582/582 [==============================] - 2s 3ms/step - loss: 0.1049 - precision: 0.6238 - recall: 0.1535
Epoch 8/500
582/582 [==============================] - 2s 3ms/step - loss: 0.1037 - precision: 0.6330 - recall: 0.1640
Epoch 9/500
582/582 [===========================

In [23]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [24]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.997483
Precision: 0.984336
Recall: 0.947356
f1: 0.965492


# N=2

In [25]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
## Reshape input to be 3D [samples, timesteps, features] (format requis par LSTM)
train_LSTM_X1 = train_LSTM_X1.reshape((train_LSTM_X1.shape[0], 1, train_LSTM_X1.shape[1]))
## Recuperation des labels
train_LSTM_y1=y_train1

In [26]:
inputs=Input((1,13))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [27]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [28]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=500,batch_size=200)

Epoch 1/500
631/631 [==============================] - 4s 3ms/step - loss: 0.2082 - precision: 0.1944 - recall: 0.1086
Epoch 2/500
631/631 [==============================] - 2s 3ms/step - loss: 0.1160 - precision: 0.5221 - recall: 0.0967
Epoch 3/500
631/631 [==============================] - 2s 3ms/step - loss: 0.1126 - precision: 0.5752 - recall: 0.0906
Epoch 4/500
631/631 [==============================] - 2s 3ms/step - loss: 0.1101 - precision: 0.5935 - recall: 0.1072
Epoch 5/500
631/631 [==============================] - 2s 4ms/step - loss: 0.1080 - precision: 0.6039 - recall: 0.1237
Epoch 6/500
631/631 [==============================] - 2s 4ms/step - loss: 0.1062 - precision: 0.5950 - recall: 0.1254
Epoch 7/500
631/631 [==============================] - 2s 4ms/step - loss: 0.1048 - precision: 0.6111 - recall: 0.1482
Epoch 8/500
631/631 [==============================] - 2s 4ms/step - loss: 0.1035 - precision: 0.6417 - recall: 0.1562
Epoch 9/500
631/631 [===========================

In [29]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [30]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.997563
Precision: 0.979848
Recall: 0.954056
f1: 0.966780


# FFNN

In [31]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)

In [32]:
X_train0=np.concatenate((y_probs0,y_probs1),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((242378, 2), (242378,))

In [33]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [34]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1),axis=1)
y_pred=clf.predict(X_test)

In [35]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.998390
Precision: 0.983084
Recall: 0.973439
f1: 0.978237
